In [1]:
import logging
import tensorflow as tf
from tensorflow.examples.tutorials.mnist import input_data as mnist_input_data

import cnn_utils

In [2]:
# Setup logging.
reload(logging)
logging.basicConfig(format = "[%(asctime)s] %(message)s",
                    datefmt = "%Y-%m-%d %H:%M:%S",
                    level = logging.INFO)

# Create an interactive session.
sess = tf.InteractiveSession()

In [3]:
# Load the dataset.
dataset = mnist_input_data.read_data_sets("MNIST_data", one_hot=True)

Extracting MNIST_data/train-images-idx3-ubyte.gz
Extracting MNIST_data/train-labels-idx1-ubyte.gz
Extracting MNIST_data/t10k-images-idx3-ubyte.gz
Extracting MNIST_data/t10k-labels-idx1-ubyte.gz


In [4]:
"""Define the network."""

# Input and output placeholders.
x = tf.placeholder(tf.float32, shape=[None, 784], name = "input")
y_ = tf.placeholder(tf.float32, shape=[None, 10])

# Build the network.
keep_prob = tf.placeholder(tf.float32, name = "dropout")
y = cnn_utils.inference(x, keep_prob)
loss = cnn_utils.loss(y, y_)
accuracy = cnn_utils.accuracy(y, y_)

# Setup training.
optimizer = tf.train.AdamOptimizer(1e-4)
train_step = cnn_utils.training(loss, accuracy, optimizer)
saver = tf.train.Saver()

summary = tf.merge_all_summaries()
summary_writer = tf.train.SummaryWriter("checkpoint", sess.graph)

# Save the graph.
tf.train.write_graph(sess.graph_def, "checkpoint", "graph.pb", as_text=True)

In [5]:
# Perform the training.
sess.run(tf.initialize_all_variables())
validation_feed_dict = {
    x:dataset.validation.images,
    y_: dataset.validation.labels,
    keep_prob: 1.0
}
for i in range(20000):
    batch = dataset.train.next_batch(50)
    sess.run(train_step, feed_dict={x: batch[0], y_: batch[1], keep_prob: 0.5})
    if (i+1) % 100 == 0:
        validation_accuracy = accuracy.eval(feed_dict=validation_feed_dict)
        logging.info("step %d, validation accuracy %.2f%%" %
                     (i+1, validation_accuracy * 100))
        saver.save(sess, "checkpoint/cnn", global_step = i+1)
        
        summary_str = sess.run(summary, feed_dict=validation_feed_dict)
        summary_writer.add_summary(summary_str, i+1)
        summary_writer.flush()

[2016-09-19 12:23:13] step 100, validation accuracy 82.36%
[2016-09-19 12:23:36] step 200, validation accuracy 90.44%
[2016-09-19 12:23:56] step 300, validation accuracy 92.50%
[2016-09-19 12:24:16] step 400, validation accuracy 93.62%
[2016-09-19 12:24:34] step 500, validation accuracy 94.86%
[2016-09-19 12:24:54] step 600, validation accuracy 95.42%
[2016-09-19 12:25:12] step 700, validation accuracy 95.34%
[2016-09-19 12:25:31] step 800, validation accuracy 95.88%
[2016-09-19 12:25:51] step 900, validation accuracy 96.12%
[2016-09-19 12:26:10] step 1000, validation accuracy 96.50%
[2016-09-19 12:26:29] step 1100, validation accuracy 96.20%
[2016-09-19 12:26:48] step 1200, validation accuracy 96.78%
[2016-09-19 12:27:07] step 1300, validation accuracy 96.98%
[2016-09-19 12:27:26] step 1400, validation accuracy 96.90%
[2016-09-19 12:27:45] step 1500, validation accuracy 97.28%
[2016-09-19 12:28:03] step 1600, validation accuracy 97.38%
[2016-09-19 12:28:22] step 1700, validation accur

In [6]:
# Run inference and get accuracy.
print("test accuracy %g"%accuracy.eval(feed_dict={
    x: dataset.test.images, y_: dataset.test.labels, keep_prob: 1.0}))

test accuracy 0.9929


Copy the `checkpoint` folder to `trained-models/cnn`.